In [1]:
import copy
import numpy as np
from persor import BVHparser
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

In [2]:
def get_joint_coords(skeleton, joint, coords):
    offset = skeleton[joint]["offset"]
    parent_joint = skeleton[joint]["joint"]
    child_joints = skeleton[joint]["children"]

    if parent_joint == None:
        return

    parent_coord = coords[parent_joint]["coord"]

    current_coord = [
        parent_coord[0] + offset[0],
        parent_coord[1] + offset[1],
        parent_coord[2] + offset[2],
    ]
    coords[joint] = {"coord": current_coord, "parent": parent_joint}

    for child in child_joints:
        get_joint_coords(skeleton, child, coords)


def skelton2coords(skeleton):
    coords = {"root": {"coord": skeleton["root"]["offset"], "parent": None}}

    child_joints = skeleton["root"]["children"]
    for child in child_joints:
        get_joint_coords(skeleton, child, coords)

    get_joint_coords(skeleton, "root", coords)

    return coords


def get_rotate_vec(frame, path):
    x_rotate = sum([frame[f"{joint}_Xrotation"] for joint in path])
    y_rotate = sum([frame[f"{joint}_Yrotation"] for joint in path])
    z_rotate = sum([frame[f"{joint}_Zrotation"] for joint in path])
    x_position = frame[f"{path[0]}_Xposition"]
    y_position = frame[f"{path[0]}_Yposition"]
    z_position = frame[f"{path[0]}_Zposition"]

    R_x = Rotation.from_euler("x", x_rotate, degrees=True).as_matrix()
    R_y = Rotation.from_euler("y", y_rotate, degrees=True).as_matrix()
    R_z = Rotation.from_euler("z", z_rotate, degrees=True).as_matrix()

    return R_y @ R_x @ R_z @ np.array([x_position, y_position, z_position])


def get_path2root(skeleton, joint):
    path = []
    while True:
        path.append(joint)
        joint = skeleton[joint]["joint"]
        if joint == None:
            break
    return path


def rotated_skeleton(skeleton, frame):
    skeleton_copy = copy.deepcopy(skeleton)

    for joint in skeleton.keys():
        if joint.startswith("_"):
            continue

        path = get_path2root(skeleton, joint)
        skeleton_copy[joint]["offset"] = get_rotate_vec(frame, path)

    return skeleton_copy

In [3]:
bvhp = BVHparser("./data/iroiro.bvh")
df = bvhp.get_motion_df()
skeleton = bvhp.get_skeleton()

In [5]:
%matplotlib qt

frame = df.iloc[1320]
skeleton_rotated = rotated_skeleton(skeleton, frame)
coords = skelton2coords(skeleton_rotated)

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(projection="3d")

for c in coords.values():
    ax.scatter(c["coord"][0], c["coord"][2], c["coord"][1], color="blue")
    parent = c["parent"]

    if parent == None:
        continue

    parent_coord = coords[parent]["coord"]
    plt.plot(
        [c["coord"][0], parent_coord[0]],
        [c["coord"][2], parent_coord[2]],
        [c["coord"][1], parent_coord[1]],
        color="black",
    )

ax.set_xlim(-100, 100)
ax.set_ylim(-100, 100)
ax.set_zlim(-20, 180)
ax.set_xlabel("X")
ax.set_ylabel("Z")
ax.set_zlabel("Y")
fig.show()
